In [ ]:
import os
import shutil
import json
from pathlib import Path
import cv2

# Load the labels from the JSON file
labels_path = '/home/alexis/Downloads/BELOW_LABELS.json'
with open(labels_path) as f:
    labels = json.load(f)

# Create directories for YOLO format labels
Path('/home/alexis/Desktop/YODOL/labels/train').mkdir(parents=True, exist_ok=True)
Path('/home/alexis/Desktop/YODOL/labels/val').mkdir(parents=True, exist_ok=True)
Path('/home/alexis/Desktop/YODOL/images/train').mkdir(parents=True, exist_ok=True)
Path('/home/alexis/Desktop/YODOL/images/val').mkdir(parents=True, exist_ok=True)

# Split data into train and val sets (assuming a 80-20 split)
image_files = list(labels.keys())
split_index = int(0.8 * len(image_files))
train_images = image_files[:split_index]
val_images = image_files[split_index:]

def convert_polyline_to_bbox(points_x, points_y, width, height):
    x_min, x_max = min(points_x), max(points_x)
    y_min, y_max = min(points_y), max(points_y)
    x_center = (x_min + x_max) / 2 / width
    y_center = (y_min + y_max) / 2 / height
    bbox_width = (x_max - x_min) / width
    bbox_height = (y_max - y_min) / height
    return f"0 {x_center} {y_center} {bbox_width} {bbox_height}"

def save_labels(images, split):
    for image in images:
        img_path = f"/home/alexis/Downloads/BELOW/{labels[image]['filename']}"
        new_image_path = f"/home/alexis/Desktop/YODOL/images/{split}/{labels[image]['filename']}"
        if not os.path.exists(img_path):
            print(f"File does not exist: {img_path}")
            continue

        try:
            img = cv2.imread(img_path)
            if img is None:
                raise Exception("Unable to read image.")
            height, width, _ = img.shape
            shutil.copy(img_path, new_image_path )
        except Exception as e:
            print(f"Error processing image: {img_path} - {e}")
            continue
        
        yolo_labels = []
        for region in labels[image]['regions']:
            points_x = region['shape_attributes']['all_points_x']
            points_y = region['shape_attributes']['all_points_y']
            yolo_labels.append(convert_polyline_to_bbox(points_x, points_y, width, height))

        label_path = f"/home/alexis/Desktop/YODOL/labels/{split}/{Path(image).stem}.txt"
        with open(label_path, 'w') as f:
            f.write("\n".join(yolo_labels))

save_labels(train_images, 'train')
save_labels(val_images, 'val')